In [3]:
#load needed libraries
from copy import deepcopy
import urllib

In [4]:
def getTexts(pmc_abstract):
    '''find open access files, extract results and methods sections and convert xml to json format'''
    for k, v in deepcopy(pmc_abstract).items():
        pmc = pmc_abstract[k]['PMC']
        if len(pmc)>0:
            pmc_idno =(s.strip('PMC') for s in v['PMC'])
            #confirm that file is Open Access
            try_this = pmc_abstract[k]['PMC']
            find_pdf = "https://www.ncbi.nlm.nih.gov/pmc/utils/oa/oa.fcgi?id={}".format(*try_this)
            #get xml record
            url = "https://www.ncbi.nlm.nih.gov/pmc/oai/oai.cgi?verb=GetRecord&identifier=oai:pubmedcentral.nih.gov:{}&metadataPrefix=pmc".format(*pmc_idno)
            pmc_abstract[k]['OA web address'] = url
            # retreive the text and pass it to the variable dic
            with urllib.request.urlopen(find_pdf) as response:
                the_file = response.read().decode('utf-8')
                dict_file = xmltodict.parse(the_file)
    
                try:
                    pmc_abstract['ftp_record'] = dict_file['OA']['records']['record']['link']['@href']
                except KeyError:
                    pmc_abstract['ftp_record'] = dict_file['OA']['error']['#text']
                except TypeError:
                    pmc_abstract['ftp_record'] = dict_file['OA']['records']['record']

            with urllib.request.urlopen(url) as responsec:
                the_filec = responsec.read().decode('utf-8')
                dict_filec = xmltodict.parse(the_filec)
         
                try:
                    data_level = dict_filec['OAI-PMH']['GetRecord']['record']['metadata']['article']['body']['sec']

                    methods = []
                    for m_level in data_level:
                        if type(m_level['title']) == str:
                            if m_level['title'].lower() in {'materials and methods','experimentalprocedures','experimental procedures','materialsandmethods', 'methods', 'method'}:
                                methods.append(m_level['sec'])
                        else:
                            methods = m_level['title']['ETHODS']

                    results = [r_level['sec'] for r_level in data_level if r_level['title'].lower()=='results']
                    pmc_abstract[k]['Methods'] = methods
                    pmc_abstract[k]['Results'] = results
                except (KeyError, TypeError):
                    continue            

    return pmc_abstract